In [85]:
import numpy as np
import pandas as pd

training_data = pd.read_csv('data/training_data.csv')
training_data = training_data.loc[training_data['tp1'] >= 0.0]
print(training_data.shape)
training_data.head()

(889472, 17)


,year,month,shop_id,shop_t,shop_tm1,shop_tm11,item_category_id,categ_t,categ_tm1,categ_tm11,item_id,t,tm1,tm2,tm5,tm11,tp1
0,2014,0,2,890.0,1322.0,488.0,40,76.0,93.0,40.0,32,1.0,0.0,0.0,0.0,0.0,0.0
1,2014,0,2,890.0,1322.0,488.0,37,44.0,55.0,21.0,33,1.0,1.0,2.0,0.0,0.0,0.0
2,2014,0,2,890.0,1322.0,488.0,37,44.0,55.0,21.0,99,1.0,0.0,0.0,0.0,0.0,0.0
3,2014,0,2,890.0,1322.0,488.0,73,4.0,3.0,10.0,482,2.0,1.0,2.0,0.0,1.0,1.0
4,2014,0,2,890.0,1322.0,488.0,73,4.0,3.0,10.0,485,1.0,1.0,0.0,0.0,0.0,1.0


In [86]:
y = training_data['tp1']

#features_to_use = ['year','month','shop_id','item_category_id','item_id','t']
#categ_features = ['year','month','shop_id','item_category_id','item_id']
#features_to_use = ['year','month','shop_id','item_category_id','t','tm1','tm2','tm5','tm11']
#categ_features = ['year','month','shop_id','item_category_id']
#features_to_use = ['year','month','shop_id','categ_t','categ_tm1','categ_tm11','t','tm1','tm2','tm5','tm11']
#categ_features = ['year','month','shop_id']
#features_to_use = ['year','month','shop_t','shop_tm1','shop_tm11','categ_t','categ_tm1','categ_tm11','t','tm1','tm2','tm5','tm11']
#categ_features = ['year','month']
#features_to_use = ['year','month','shop_t','shop_tm1','shop_tm11','item_category_id','t','tm1','tm2','tm5','tm11']
#categ_features = ['year','month','item_category_id']
features_to_use = ['year','month','shop_id','item_category_id','item_id','t','tm1','tm2','tm5','tm11']
categ_features = ['year','month','shop_id','item_category_id','item_id']
X = training_data[features_to_use]
X.head()

,year,month,shop_id,item_category_id,item_id,t,tm1,tm2,tm5,tm11
0,2014,0,2,40,32,1.0,0.0,0.0,0.0,0.0
1,2014,0,2,37,33,1.0,1.0,2.0,0.0,0.0
2,2014,0,2,37,99,1.0,0.0,0.0,0.0,0.0
3,2014,0,2,73,482,2.0,1.0,2.0,0.0,1.0
4,2014,0,2,73,485,1.0,1.0,0.0,0.0,0.0


In [87]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Validation set has {} samples.".format(X_valid.shape[0]))

Training set has 711577 samples.
Validation set has 177895 samples.


In [55]:
from sklearn.metrics import mean_squared_error

def calc_RMSE(actuals, predictions):
    return np.sqrt(mean_squared_error(actuals, predictions))

def calc_RMSE_cb(actuals, predictions):
    return 'RMSE', np.sqrt(mean_squared_error(actuals, predictions)), False

In [45]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

def search_for_best_model(inputs, targets):
    cv_sets = ShuffleSplit(n_splits=10, test_size=0.2, train_size=None, random_state=42)
    regressor = LGBMRegressor(learning_rate=0.1, n_estimators=150, reg_lambda=0.0001, n_jobs=4)
    #params = {'max_depth':[50, 75, 100],'num_leaves':[1250, 1500, 1750], learning_rate:[0.1, 0.01, 0.001]}
    #params = {'max_depth':[50, 75, 100],'num_leaves':[1250, 1500, 1750], n_estimators:[100, 150, 200]}
    params = {'max_depth':[70, 75, 80],'num_leaves':[1200, 1250, 1300]}
    scoring_fnc = make_scorer(calc_RMSE, greater_is_better=False)
    grid = GridSearchCV(regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)
    grid = grid.fit(inputs, targets, categorical_feature=categ_features)
    return grid.best_estimator_

#best_model = search_for_best_model(X_train, y_train)
#print("Parameter 'max_depth' is {} for the optimal model.",format(best_model.get_params()['max_depth']))
#print("Parameter 'num_leaves' is {} for the optimal model.",format(best_model.get_params()['num_leaves']))
#print("Parameter 'learning_rate' is {} for the optimal model.",format(best_model.get_params()['learning_rate']))
#print("Parameter 'n_estimators' is {} for the optimal model.",format(best_model.get_params()['n_estimators']))

In [88]:
regressor = LGBMRegressor(max_depth=80, num_leaves=1500, learning_rate=0.1, n_estimators=150, \
    reg_lambda=0.0001, n_jobs=4)
eval_set=[(X_valid, y_valid)]
regressor_model = regressor.fit(X_train, y_train, categorical_feature=categ_features, \
    eval_set=eval_set, eval_metric=calc_RMSE_cb, early_stopping_rounds=5)

predictions_train = regressor_model.predict(X_train)
print('Training RMSE:', calc_RMSE(y_train, predictions_train))

/home/dylbert/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1042: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['item_category_id', 'item_id', 'month', 'shop_id', 'year']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's l2: 41.2016	valid_0's RMSE: 6.41884
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 36.5578	valid_0's RMSE: 6.04631
[3]	valid_0's l2: 32.7595	valid_0's RMSE: 5.72359
[4]	valid_0's l2: 29.4235	valid_0's RMSE: 5.42435
[5]	valid_0's l2: 26.7518	valid_0's RMSE: 5.17221
[6]	valid_0's l2: 24.2327	valid_0's RMSE: 4.92267
[7]	valid_0's l2: 22.2458	valid_0's RMSE: 4.71655
[8]	valid_0's l2: 20.8022	valid_0's RMSE: 4.56095
[9]	valid_0's l2: 19.4531	valid_0's RMSE: 4.41057
[10]	valid_0's l2: 18.4171	valid_0's RMSE: 4.29151
[11]	valid_0's l2: 17.4931	valid_0's RMSE: 4.18247
[12]	valid_0's l2: 16.7636	valid_0's RMSE: 4.09433
[13]	valid_0's l2: 16.2456	valid_0's RMSE: 4.03058
[14]	valid_0's l2: 15.8699	valid_0's RMSE: 3.9837
[15]	valid_0's l2: 15.3875	valid_0's RMSE: 3.92268
[16]	valid_0's l2: 15.1892	valid_0's RMSE: 3.89734
[17]	valid_0's l2: 15.0206	valid_0's RMSE: 3.87564
[18]	valid_0's l2: 14.8326	valid_0's RMSE: 3.85131
[19]	valid_0's l2: 14.6443	vali

In [89]:
test_file = pd.read_csv('data/provided/test.csv')
print(test_file.shape)
test_file.head()

(214200, 3)


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [90]:
#get previous year's sales totals
test_file['date_block_num'] = 33
monthly_totals_all = pd.read_csv('data/td_all.csv')

test_data = test_file.merge(monthly_totals_all, on=['date_block_num','shop_id','item_id'], how='left')
print(test_data.shape)
test_data.head()

(214200, 19)


,ID,shop_id,item_id,date_block_num,t,tm1,tm2,tm5,tm11,tp1,item_category_id,categ_t,categ_tm1,categ_tm11,shop_t,shop_tm1,shop_tm11,year,month
0,0,5,5037,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5,5320,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,5233,33,1.0,3.0,1.0,3.0,0.0,0.0,19.0,75.0,110.0,77.0,1052.0,1092.0,1445.0,2015.0,4.0
3,3,5,5232,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,5268,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
test_data['year'] = 2015
test_data['month'] = 9
test_data.fillna(0.0, inplace=True)
test_data = test_data[features_to_use]
test_data.head()

,year,month,shop_id,item_category_id,item_id,t,tm1,tm2,tm5,tm11
0,2015,9,5,0.0,5037,0.0,0.0,0.0,0.0,0.0
1,2015,9,5,0.0,5320,0.0,0.0,0.0,0.0,0.0
2,2015,9,5,19.0,5233,1.0,3.0,1.0,3.0,0.0
3,2015,9,5,0.0,5232,0.0,0.0,0.0,0.0,0.0
4,2015,9,5,0.0,5268,0.0,0.0,0.0,0.0,0.0


In [92]:
preds_submissions = regressor_model.predict(test_data)

submissions = pd.DataFrame({
    "ID": test_file["ID"],
    "item_cnt_month": preds_submissions.clip(0., 20.)
})
submissions.to_csv("data/submission.csv", index=False)

First submission: score of 1.23840 

First clean submission: score of 1.21787